In [1]:
! pip install langchain-chroma # pour le vector database

  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
     ---------------------------------------- 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 526.8/526.8 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 150.6/150.6 kB 1.8 MB/s eta 0:00:00
   ---------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.13 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.2.5 which is incompatible.
langchain-community 0.0.29 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.2.5 which is incompatible.
langchain-text-splitters 0.0.1 requires langchain-core<0.2.0,>=0.1.28, but you have langchain-core 0.2.5 which is incompatible.


In [2]:
! pip install langchain-core sentence-transformers

In [3]:
! pip install chromadb

In [4]:
! pip install pypdf

   ---------------------------------------- 290.4/290.4 kB 2.2 MB/s eta 0:00:00


In [7]:
! pip install langchain-community

   ---------------------------------------- 302.9/302.9 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.5
    Uninstalling langchain-core-0.2.5:
      Successfully uninstalled langchain-core-0.2.5


langchain-chroma # pour le vector database  \
langchain-core # pour les parseurs \
sentences-transformers # pour les modèles d'embedding \
chromadb # pour le vector databse \
pypdf # pour parser les pdf

In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader('./code_civil.pdf')

In [3]:
pages = loader.load_and_split()

In [4]:
for page in pages[:3]:
    print(page.metadata)

{'source': './code_civil.pdf', 'page': 0}
{'source': './code_civil.pdf', 'page': 1}
{'source': './code_civil.pdf', 'page': 2}


In [5]:
pages_filtered = list(filter(lambda page: (p := page.metadata['page']) >= 12 and p<=655, pages))

In [6]:
pages_filtered[13]

Document(page_content="Livre Ier : Des personnes - Titre Ier bis : De la nationalité française - Chapitre II : De la nationalité française d'origine\n19-3  Ordonnance n°2005-759 du 4 juillet 2005 - art. 17 () JORF 6 juillet 2005 en vigueur le 1er juillet 2006      \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nEst français l'enfant né en France lorsque l'un de ses parents au moins y est lui-même né.\n19-4  Loi n°93-933 du 22 juillet 1993 - art. 50 () JORF 23 juillet 1993      \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nToutefois, si un seul des parents est né en France, l'enfant français, en vertu de l'article 19-3, a la faculté de\nrépudier cette qualité dans les six mois précédant sa majorité et dans les douze mois la suivant.\nCette faculté se perd si l'un des parents acquiert la nationalité française durant la minorité de l'enfant.\nSection 3 : Dispositions communes\n20  Loi n°93-933 du 22 juillet 1993 - art. 50 () 

In [7]:
from typing import List, Dict, Any

In [8]:
class Paragraph:
    def __init__(self,
                 number: int,
                 title: str,
                 start_page: int,
                 end_page: int,
                 metadata: Dict[str, Any]):
        self.text = text
        self.metadata = metadata

In [9]:
# class Section:

#     def __init__(self,
#                  number: int,
#                  title: str,
#                  paragraphs: 
                 
#                  metadata: Dict[str, Any], content: str):
#         self.metadata = metadata
#         self.content = content

In [10]:
class Chapitre:

    def __init__(self,
                 number: int,
                 title: str,
                 first_page: int,
                 last_page:int,
                 sections: List[Dict[str, Any]]):
        
        self.title = title
        self.number = number
        self.sections = sections

In [13]:
class Livre:
    def __init__(self, number: int, title: str, first_page: int, last_page: int, children: List[Chapitre] = None):
        self.number = number
        self.title = title
        self.first_page = first_page
        self.last_page = last_page
        self.children = children or []

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                              chunk_overlap=50,
                                              length_function=len)


In [16]:
doc_splitted = splitter.split_documents(pages_filtered)

In [17]:
len(doc_splitted)

13426

In [18]:
from langchain_chroma import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [20]:
embedding_model = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-small',
    model_kwargs={'device': 'mps'},
    encode_kwargs={'normalize_embeddings': True},
    cache_folder= '.'
)

In [21]:
chroma = Chroma.from_documents(doc_splitted,
                               embedding_model,
                               persist_directory='./chroma-db')

In [22]:
chroma

In [32]:
! pip install cohere

  Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl.metadata (6.8 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 165.1/165.1 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 139.3/139.3 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 497.2/497.2 kB 5.2 MB/s eta 0:00:00
Using cached tokenizers-0.15.2-cp310-none-win_amd64.whl (2.2 MB)
   ---------------------------------------- 12.3/12.3 MB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 82.2/82.2 kB 2.3 MB/s eta 0:00:00
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.40.1 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


In [23]:
chroma.similarity_search_with_score('droit de la famille', 5)

[(Document(page_content="du droit à l'image de l'enfant.\nDans tous les cas visés au présent article, les deux parents doivent être appelés à l'instance. Lorsque l'enfant", metadata={'page': 166, 'source': './code_civil.pdf'}),
  0.20696404576301575),
 (Document(page_content="faire obstacle à l'exercice de ce droit.\nSi tel est l'intérêt de l'enfant, le juge aux affaires familiales fixe les modalités des relations entre l'enfant et un", metadata={'page': 153, 'source': './code_civil.pdf'}),
  0.21113593876361847),
 (Document(page_content="leurs droits d'autorité parentale, le consentement est donné par le conseil de famille, après avis de la personne", metadata={'page': 140, 'source': './code_civil.pdf'}),
  0.21195100247859955),
 (Document(page_content='Dictionnaire du Droit privé\n> Autorité parentale\n> Juge aux affaires familiales (JAF)\n> Bien Propre\n> Séparation de biens\nRessources\n> Les modes de saisine du JAF', metadata={'page': 95, 'source': './code_civil.pdf'}),
  0.212706

In [25]:
# from langchain.llms import HuggingFacePipeline
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
# from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [26]:
llm = Ollama(
    model="llama2"
)

In [27]:
# text_gen_pipeline = pipeline('text-generation',
#                             device='cuda')

In [28]:
# llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

In [30]:
propmt_template = """
Réponds en français. Avec l'aide de ces informations provenant du code civil français, réponds à la question.
Contexte: {context}
Question: {question}
"""

In [31]:
prompt = PromptTemplate(template=propmt_template,
                        input_variables=['context', 'question'])

In [32]:
def prepare_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [33]:
retriver = chroma.as_retriever()

In [34]:
chain = (
    {'context': retriver | prepare_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x3a1084c10>)
           | RunnableLambda(prepare_docs),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], template="\nAvec l'aide de ces informations provenant du code civil français, réponds à la question.\nContexte: {context}\nQuestion: {question}\n")
| Ollama()
| StrOutputParser()

In [36]:
chain.invoke('Quel est le droit de la famille en France?')

"Le droit de la famille en France est governé par le Code civil français. According to Article 418 of the Civil Code, the family is a fundamental unit of society and has the right to protection and support from the state. This article provides for the recognition and protection of the rights of spouses, children, and other members of the family, as well as the duties and responsibilities of each member towards the others.\n\nIn France, the legal system recognizes several types of families, including:\n\n1. Marriage: A marriage between two consenting adults is recognized by French law. The spouses have mutual rights and obligations, including the right to inherit from each other and the obligation to support each other.\n2. PACS (Pacte Civil de Solidarité): A PACS is a civil solidarity pact between two adults who live together in a stable relationship. It provides for mutual rights and obligations, including the right to inherit from each other and the obligation to support each other.\

In [38]:
import os
import cohere

from typing import List, Tuple
import gradio as gr

class AIJurist:

    def __init__(self,
                 embedding_model_path: str,
                 vector_db_path: str,
                 sys_prompt_path: str,
                 top_k: int = 5,
                 embedding_model_name: str = os.getenv('EMBEDDING_MODEL_NAME'),
                 api_key: str = os.getenv('CO_API_KEY'),
                 device: str = 'cpu',
                 **kwargs):
        
        self.cohere_client = cohere.Client(api_key=api_key)

        self.embedding_model = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            model_kwargs={'device': device},
            encode_kwargs={'normalize_embeddings': True},
            cache_folder=embedding_model_path
        )

        self.vdb = Chroma(
            persist_directory=vector_db_path,
            embedding_model=self.embedding_model
        )

        self.top_k = top_k

        with open(sys_prompt_path) as f:
            self.sys_prompt = f.read()

        self.cohere_params = kwargs

    def _format_chat_history(self,
                            history: List[Tuple[str, str]]) -> List[Dict[str, str]]:

        chat_history = []

        for user_message, assistant_message in history:
            chat_history.append({'role': 'USER', 'message': user_message})
            chat_history.append({'role': 'CHATBOT', 'message': assistant_message})

        return chat_history

    def _get_response(self,
                      message: str,
                      history: List[Tuple[str, str]]):
        
        documents = self.vdb.similarity_search(message, self.top_k)

        context ="\n".join([doc.page_content for doc in documents])

        message_augmented = f"{self.sys_prompt}Avec l'aide de ces extraits du code civil français: {context} réponds à la question suivante: {message}"

        chat_history = self._format_chat_history(history) if history else []

        response = self.cohere_client.chat_stream(message=message_augmented,
                                                  chat_history=chat_history,
                                                  **self.cohere_params)
        
        full_response = ""
        for event in response:
            if event.event_type == "text-generation":
                full_response += event.text
                yield full_response

    def deploy_chatbot(self,
                       title: str,
                       description: str) -> None:
        
        gr.ChatInterface(fn=self._get_response,
                         title=title,
                         description=description).launch()